In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [3]:
df = pd.read_excel("Demo.xlsx")
df['Date Of Joining'] = pd.to_datetime(df['Date Of Joining']).dt.strftime('%Y-%m-%d')
columns_to_convert = ['Gross Salary', 'Bonus / Commission', 'Increment', 
                      'Reimbursment Amount', 'Compensation', 'Adjustments', 
                      'Absents Deduction', 'Lates Deduction']

for col in columns_to_convert:
    df[col] = df[col].astype(int)


In [17]:
driver = webdriver.Edge()
driver.get('https://printyourcopy.com/free-payslip-generator')
time.sleep(1)

#Clearing Function
def clear_values():

    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(Keys.TAB)  
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.TAB)
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.3)

    driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.TAB) 

    driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.TAB) 

#Logo
logo = os.path.abspath("Logo.jpg")
file_input = driver.find_element(By.XPATH, '//*[@id="logo"]').send_keys(logo)

#payslip no.
driver.find_element(By.XPATH, '//*[@id="serial"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="serial"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="serial"]').send_keys(Keys.TAB)   

#Eployee/Employer signature clearing
driver.find_element(By.XPATH, '//*[@id="employeeSign"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="employeeSign"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="employeeSign"]').send_keys(Keys.TAB)   
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="employerSign"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="employerSign"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="employerSign"]').send_keys(Keys.TAB)   

#Employer Details
employer_details = f"""
Robust Support & Solutions
Office No.404A, Fortune Tower
PECHS Block 6, Karachi, Pakistan
Phone & WhatsApp: 0311-3859635
"""
driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(employer_details)

#Currency Input
driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys("PKR")
driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys(Keys.TAB)   

time.sleep(2)
# Earning boxes
n = 4
for i in range(4):
    wait = WebDriverWait(driver,5)
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button')))
    button.click()
    time.sleep(1)
    n+=1

time.sleep(2)
# Deduction boxes
n = 11
for i in range(3):
    wait = WebDriverWait(driver,5)
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button')))
    button.click()
    time.sleep(1)
    n+=1

time.sleep(1)
# Basic pay desc
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys("Gross Salary")
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Bonus / Commission desc 
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys("Bonus / Commission")
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Reimbursement Amount desc
driver.find_element(By.XPATH, '//*[@id="earnings.2.description"]').send_keys("Reimbursement")
driver.find_element(By.XPATH, '//*[@id="earnings.2.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Increment desc
driver.find_element(By.XPATH, '//*[@id="earnings.3.description"]').send_keys("Increment")
driver.find_element(By.XPATH, '//*[@id="earnings.3.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Compensation desc
driver.find_element(By.XPATH, '//*[@id="earnings.4.description"]').send_keys("Compensation")
driver.find_element(By.XPATH, '//*[@id="earnings.4.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Adjustments desc
driver.find_element(By.XPATH, '//*[@id="earnings.5.description"]').send_keys("Adjustments")
driver.find_element(By.XPATH, '//*[@id="earnings.5.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Absents desc
driver.find_element(By.XPATH, '//*[@id="deductions.0.description"]').send_keys(Keys.CONTROL + "a")  # Select all text
driver.find_element(By.XPATH, '//*[@id="deductions.0.description"]').send_keys(Keys.DELETE)        # Clear it
driver.find_element(By.XPATH, '//*[@id="deductions.0.description"]').send_keys("Absents")
driver.find_element(By.XPATH, '//*[@id="deductions.0.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Lates desc 
driver.find_element(By.XPATH, '//*[@id="deductions.1.description"]').send_keys("Lates")
driver.find_element(By.XPATH, '//*[@id="deductions.1.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Payroll Tax desc
driver.find_element(By.XPATH, '//*[@id="deductions.2.description"]').send_keys("Payroll Tax")
driver.find_element(By.XPATH, '//*[@id="deductions.2.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

# Others desc
driver.find_element(By.XPATH, '//*[@id="deductions.3.description"]').send_keys("Others")
driver.find_element(By.XPATH, '//*[@id="deductions.3.description"]').send_keys(Keys.TAB)   
time.sleep(0.3)

for i in range(2):
    # Employee Details
    employee_details= f"""{df.loc[i, 'Full Name (As Per CNIC)']}
    {df.loc[i, 'CNIC Number']}
    {df.loc[i, 'Designation']}
    DOJ: {df.loc[i, 'Date Of Joining']}
    """
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(employee_details)

    driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(str(df.loc[i, 'Gross Salary']))
    driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(str(df.loc[i, 'Bonus / Commission']))
    driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(str(df.loc[i, 'Reimbursment Amount']))
    driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(str(df.loc[i, 'Increment']))
    driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(str(df.loc[i, 'Compensation']))
    driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(str(df.loc[i, 'Adjustments']))
    driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(str(df.loc[i, 'Absents Deduction']))
    driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(str(df.loc[i, 'Lates Deduction']))
    driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.TAB)  
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(str(df.loc[i, 'Payroll Tax Deduction']))
    driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(str(df.loc[i, 'Other Deductions']))
    driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.TAB) 
    time.sleep(0.5)

    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[2]/button[2]').click()
    time.sleep(0.5)
    
    clear_values()
    time.sleep(0.5)
    n+=1


In [ ]:
    # if i == 0:
    #     original_file = r"C:\Users\Desktop\Downloads\Payslip.pdf"
    # else:
    #     original_file = fr"C:\Users\Desktop\Downloads\Payslip ({i}).pdf"
    
    # # New file path with employee's name
    # new_file = fr'{df.loc[i, "Full Name (As Per CNIC)"]}.pdf'

    # # Rename the file
    # os.rename(original_file, new_file)

    # print(f"Renamed: {original_file} -> {new_file}")